# RAG Usando o Serviço de Agente do Azure AI e Semantic Kernel

Este trecho de código demonstra como criar e gerenciar um agente do Azure AI para geração aumentada por recuperação (RAG) usando o `Serviço de Agente do Azure AI` e o `Semantic Kernel`. O agente processa consultas dos usuários com base no contexto recuperado e fornece respostas precisas de acordo.


Correção de Versão do SQLite  
Se você encontrar o erro:  
```
RuntimeError: Your system has an unsupported version of sqlite3. Chroma requires sqlite3 >= 3.35.0
```  

Descomente este bloco de código no início do seu notebook:  


In [ ]:
# %pip install pysqlite3-binary
# __import__('pysqlite3')
# import sys
# sys.modules['sqlite3'] = sys.modules.pop('pysqlite3')

### Importando Pacotes
O código a seguir importa os pacotes necessários:


In [ ]:
# Azure imports for project client and credentials
from azure.ai.projects.models import FileSearchTool, OpenAIFile, VectorStore
from azure.identity.aio import DefaultAzureCredential

# Semantic Kernel imports
from semantic_kernel.agents import AzureAIAgent, AzureAIAgentThread

# Geração Aumentada por Recuperação com Semantic Kernel & Azure AI Agent Service

Este exemplo demonstra como usar o **Azure AI Agent Service** para realizar **Geração Aumentada por Recuperação (RAG)**, combinando um modelo de linguagem com contexto específico de domínio a partir de um documento enviado.

### Como Funciona

1. **Envio de Documento**: Um arquivo markdown (document.md) contendo informações (política de seguro de viagem da Contoso) é enviado ao serviço de agente.

2. **Criação de Armazenamento Vetorial**: O documento é indexado em um armazenamento vetorial para permitir busca semântica sobre seu conteúdo.

3. **Configuração do Agente**: Um agente é instanciado usando o modelo `gpt-4o` com as seguintes instruções rigorosas:
   - Responder apenas perguntas baseadas no conteúdo recuperado do documento.
   - Recusar-se a responder se a pergunta estiver fora do escopo.

4. **Integração da Ferramenta de Busca de Arquivos**: A `FileSearchTool` é registrada com o agente, permitindo que o modelo busque e recupere trechos relevantes do documento indexado durante a inferência.

5. **Interação com o Usuário**: Os usuários podem fazer perguntas. Se informações relevantes forem encontradas no documento, o agente gera uma resposta fundamentada.  
   Caso contrário, o agente responde explicitamente que o documento não contém informações suficientes.


### Função Principal

Esta é a função principal do programa. Ela é responsável por inicializar os componentes e gerenciar o fluxo principal da aplicação.

```python
def main():
    # Inicializa os componentes necessários
    initialize_components()

    # Executa o fluxo principal
    run_main_flow()
```

### Configuração

Certifique-se de configurar corretamente os parâmetros antes de executar o programa. Isso inclui definir variáveis de ambiente e ajustar as configurações no arquivo de configuração.

### Dependências

Antes de começar, instale todas as dependências necessárias. Você pode fazer isso executando o seguinte comando:

```bash
pip install -r requirements.txt
```

### Executando o Programa

Para executar o programa, use o seguinte comando:

```bash
python main.py
```

[!NOTE] Certifique-se de que todas as dependências foram instaladas antes de executar o programa.

### Solução de Problemas

Se você encontrar problemas ao executar o programa, verifique os seguintes pontos:

1. Certifique-se de que todas as dependências estão instaladas corretamente.
2. Verifique se as variáveis de ambiente estão configuradas corretamente.
3. Consulte os logs para obter mais informações sobre o erro.

[!TIP] Use a flag `--debug` para ativar o modo de depuração e obter mais detalhes durante a execução.

### Contribuindo

Se você deseja contribuir para este projeto, siga estas etapas:

1. Faça um fork do repositório.
2. Crie um branch para suas alterações.
3. Envie um pull request com uma descrição detalhada das alterações.

[!IMPORTANT] Certifique-se de seguir as diretrizes de contribuição antes de enviar seu pull request.

### Licença

Este projeto está licenciado sob a licença MIT. Consulte o arquivo LICENSE para mais detalhes.


Certifique-se de executar primeiro `az login` usando o Azure CLI para que o contexto de autenticação adequado seja fornecido ao usar o `DefaultAzureCredential`. O Azure AI Agent Service não utiliza chaves de API.


In [ ]:
async def main():
    async with (
        DefaultAzureCredential() as creds,
        AzureAIAgent.create_client(credential=creds) as client,
    ):
        file: OpenAIFile = await client.agents.upload_file_and_poll(file_path="document.md", purpose="assistants")
        vector_store: VectorStore = await client.agents.create_vector_store_and_poll(
            file_ids=[file.id], name="my_vectorstore"
        )

        # Define agent name and instructions tailored for RAG.
        AGENT_NAME = "RAGAgent"
        AGENT_INSTRUCTIONS = """
        You are an AI assistant designed to answer user questions using only the information retrieved from the provided document(s).

        - If a user's question cannot be answered using the retrieved context, **you must clearly respond**: 
        "I'm sorry, but the uploaded document does not contain the necessary information to answer that question."
        - Do not answer from general knowledge or reasoning. Do not make assumptions or generate hypothetical explanations.
        - Do not provide definitions, tutorials, or commentary that is not explicitly grounded in the content of the uploaded file(s).
        - If a user asks a question like "What is a Neural Network?", and this is not discussed in the uploaded document, respond as instructed above.
        - For questions that do have relevant content in the document (e.g., Contoso's travel insurance coverage), respond accurately, and cite the document explicitly.

        You must behave as if you have no external knowledge beyond what is retrieved from the uploaded document.
        """

        
        # Create file search tool with uploaded resources
        file_search = FileSearchTool(vector_store_ids=[vector_store.id])

        # 3. Create an agent on the Azure AI agent service with the file search tool
        agent_definition = await client.agents.create_agent(
            model="gpt-4o",  # This model should match your Azure OpenAI deployment.
            name=AGENT_NAME,
            instructions=AGENT_INSTRUCTIONS,
            tools=file_search.definitions,
            tool_resources=file_search.resources,
        )
        
        # Create the Azure AI Agent using the client and definition.
        agent = AzureAIAgent(
            client=client,
            definition=agent_definition,
        )
        
        # Create a thread to hold the conversation
        # If no thread is provided, a new thread will be
        # created and returned with the initial response
        thread: AzureAIAgentThread | None = None
        
        # Example user queries.
        user_inputs = [
            "Can you explain Contoso's travel insurance coverage?",  # Relevant context.
            "What is a Neural Network?"  # No relevant context from the document. Will not contain a source annotation.
        ]
        
        try:
            for user_input in user_inputs:
                print(f"# User: '{user_input}'")
                # Invoke the agent for the specified thread for response
                async for response in agent.invoke(messages=user_input, thread=thread):
                    print(f"# {response.name}: {response}")
                    thread = response.thread
        finally:
            # Clean up resources.
            await thread.delete() if thread else None
            await client.agents.delete_vector_store(vector_store.id)
            await client.agents.delete_file(file.id)
            await client.agents.delete_agent(agent.id)
            print("\nCleaned up agent, thread, file, and vector store.")

await main()

Entendido! Por favor, forneça o conteúdo do arquivo Markdown que você gostaria que eu traduza para o português (Brasil).



---

**Aviso Legal**:  
Este documento foi traduzido utilizando o serviço de tradução por IA [Co-op Translator](https://github.com/Azure/co-op-translator). Embora nos esforcemos para garantir a precisão, esteja ciente de que traduções automáticas podem conter erros ou imprecisões. O documento original em seu idioma nativo deve ser considerado a fonte oficial. Para informações críticas, recomenda-se a tradução profissional realizada por humanos. Não nos responsabilizamos por quaisquer mal-entendidos ou interpretações equivocadas decorrentes do uso desta tradução.
